___

<a href='https://mainacademy.ua/'> <img src='https://mainacademy.ua/wp-content/uploads/2019/03/logo-main.png' alt = 'Хмм, щось з інтернетом'/></a>
___

# Module 4: Work with data professionally!

## Lab work 4



#### Мета: 

* навчитися обробляти дані
* "парсити" сайти

### Завдання 1:

#### Ознайомитися із API Національного банку 
https://old.bank.gov.ua/control/uk/publish/article?art_id=38441973&cat_id=38459171#exchange



1. Підключитися до API.


In [1]:
import requests

url = 'https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange?json'
params = {'valcode' : 'EUR'} 

response = requests.get(url, params=params)
print(response)

<Response [200]>


2. Отримати курс євро за останній рік.

In [2]:
import datetime as DT
import requests

annual_exchange_rate_EURO = {} # ініціалізую словник для подобового значення курсу євро

params = {'date' : '20221018', 'valcode' : 'EUR'} #задамо параметри згідно з документацією API
url = 'https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange?json' # задамо адресу ресурсу
date = DT.datetime.now() # задамо сьогоднішню датау

for day in range(366): # цикл з 365 днів
    params['date'] = (date - DT.timedelta(days=day)).strftime('%Y%m%d') # зміна дати в параметрах запиту
    try:
        response = requests.get(url, params=params) # дані GET запиту
        data = response.json()[0] # запис необхідних даних  у змінну
        annual_exchange_rate_EURO[data['exchangedate']] = data['rate'] # заповнення словника
        start_time += DT.timedelta(days=day) # перезадаємо наступну дату для запиту
    except:
        print('error')
        break

### Працює скрипт через раз, гадаю що проблеми на сервері, тому запишу отримані дані:

In [3]:
# import pickle

# with open('annual_exchange_rate_EURO.bin', 'bw') as f:
#     pickle.dump(annual_exchange_rate_EURO, f)

In [4]:
# зчитую отримані дані:
from io import BytesIO
from urllib.request import urlopen
import pickle

url = 'https://github.com/pruhlo/data-science-assignment/blob/master/assets/annual_exchange_rate_EURO.bin?raw=true'
raw = BytesIO(urlopen(url).read())
EURO = pickle.load(raw)

3. Вивести середнє значення та відхилення курсу за кожний місяць.

In [1]:
import datetime as DT
import numpy as np
import pandas as pd

In [6]:
%%time

month_EURO ={} # ініціалізуємо словник з місячними значеннями курсу євро

for key in EURO.keys():
    month = DT.datetime.strptime(key, '%d.%m.%Y').strftime("%B") # створимо ключ(місяць)
    try:
        month_EURO[month].append(float(EURO[key])) # якщо в словнику є даний список доповнимо його
    except KeyError:
        month_EURO[month] = [float(EURO[key])] # якщо в словнику не має списку створимо його
        
describe_month_EURO = {} # словник для 

for key in month_EURO.keys():
    describe_month_EURO[key + ' mean'] = np.mean(month_EURO[key])
    describe_month_EURO[key + ' std'] = np.std(month_EURO[key])

CPU times: total: 0 ns
Wall time: 9.99 ms


In [7]:
%%time

df = pd.DataFrame.from_dict({'date':EURO.keys(), 'EURO':EURO.values()})
df.date = pd.to_datetime(df.date, format='%d.%m.%Y') # якщо не вказати формат можливо буде помилка при групуванні

result = pd.DataFrame(columns=['mean', 'std'], index=list(range(1,13)))
result['mean'] = df.groupby(df.date.dt.month)['EURO'].mean()
result['std'] = df.groupby(df.date.dt.month)['EURO'].std()

CPU times: total: 15.6 ms
Wall time: 15 ms


In [83]:
# result

In [8]:
%%time

# Ще спосіб групування за допомогою pandas та datetime
DF = pd.DataFrame(columns=['data', 'month', 'EURO'])

for key in EURO:
    DF.loc[len(DF.index)] = [key, DT.datetime.strptime(key, '%d.%m.%Y').strftime("%B"), EURO[key]]

groupBy = DF.groupby('month').mean(numeric_only=True) # numeric_only=True >>> без вказування помилка
groupBy['sdt'] = DF.groupby('month').std(numeric_only=True)

CPU times: total: 797 ms
Wall time: 823 ms


4. Дану інформацію записати у файл за допомогою pickle.

In [9]:
import pickle

with open('mydata_LW-4.bin', 'bw') as f:
    pickle.dump(month_EURO, f)
    pickle.dump(describe_month_EURO, f)
    pickle.dump(df, f)
    pickle.dump(result, f)
    pickle.dump(DF, f)
    pickle.dump(groupBy, f)

In [29]:
# ПРОЧИТАТИ з репозиторію вказані дані
# url = 'https://github.com/pruhlo/data-science-assignment/blob/master/assets/mydata_LW-4.bin?raw=true'
# raw = BytesIO(urlopen(url).read())
# month_EURO1 = pickle.load(raw)
# describe_month_EURO1 = pickle.load(raw)
# df1 = pickle.load(raw)
# result1 = pickle.load(raw)
# DF1 = pickle.load(raw)
# groupBy1 = pickle.load(raw)

### Завдання 2:

Потрібно проаналізувати всі товари на сайті: 
https://smallpacking.agrosem.ua/shop/

Створіть базу даних, що містить таблицю із такими стовбцями: 
    - артикул товару
    - найменування
    - ціна 
    - вага

In [16]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

def make_req(url, page):
    
    ''' Query with nuber of page into web store
    HEADERS see: https://raw.githubusercontent.com/Lidkins/DS/915bdbd30ca3b0dd2fc507db94b41ee62cbcebf3/Lab%20work%204.ipynb'''
#     HEADERS = {
#              'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
#              'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
#              'Accept-Language': 'ua-UA,en;q=0.5',
#              'Accept-Encoding': 'gzip, deflate',
#              'DNT': '1',
#              'Connection': 'keep-alive',
#              'Upgrade-Insecure-Requests': '1',
#              'page':f'{page}'
    HEADERS = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36',
    }
#     headers = {
#         'User-Agent': 'Mozilla/5.0',
#         'page':f'{page}'}
    resp = requests.get(url, headers=HEADERS, params = {'page': f'{page}'})
    return resp

def get_data(df):
    base_select_tags = 'div.item' # базовий тег для отримання даних
    for page in range(1,30):
        url = 'https://smallpacking.agrosem.ua/' # URLs сторінок з данними
        response = make_req(url, page) # результат запиту
        soup = BeautifulSoup(response.text, 'html.parser') # об'єкт супу
        list_tags = [i.get('content') for i in soup.select(f'{base_select_tags}')] # витягаємо контент
        for _ in list_tags:
            dicty = eval(_) # отримуємо словник
            df.loc[len(df.index)] = dicty # додаємо рядок до датафрейму
    return df

df = pd.DataFrame(columns=['id', 'name', 'weight', 'price'])

df = get_data(df)
df

,id,name,weight,price
0,1604,ДОБРИВО ДЛЯ ОРХІДЕЙ,180 г,119.00
1,1609,ДОБРИВО ДЛЯ ТРОЯНД ТА КВІТУЧИХ РОСЛИН,180 г,119.00
2,1600,ДОБРИВО ДЛЯ КВІТУЧИХ КІМНАТНИХ РОСЛИН,180 г,119.00
3,1599,ДОБРИВО ДЛЯ ДЕКОРАТИВНО-ЛИСТЯНИХ КІМНАТНИХ РОСЛИН,180 г,119.00
4,1522,ДОБРИВО ДЛЯ ДЕКОРАТИВНИХ ЛИСТОВИХ,3 кг,499.00
...,...,...,...,...
1039,1600,ДОБРИВО ДЛЯ КВІТУЧИХ КІМНАТНИХ РОСЛИН,180 г,119.00
1040,1599,ДОБРИВО ДЛЯ ДЕКОРАТИВНО-ЛИСТЯНИХ КІМНАТНИХ РОСЛИН,180 г,119.00
1041,1608,ДОБРИВО ДЛЯ САМШИТУ ТА ВІЧНОЗЕЛЕНИХ ЧАГАРНИКІВ,180 г,119.00
1042,1610,ДОБРИВО ДЛЯ ХВОЙНИХ РОСЛИН,180 г,129.00


In [18]:
df.id.nunique()

26

### Завдання 3: 

Ознайомтеся із роботою SQLite та відповідним модулем у Python.
Завантажте базу даних для виконання лабораторної роботи.
Підключіться до завантаженої бази SQLite.

1. Виведіть інформацію про дану базу.

In [19]:
import sqlite3

# Create a SQL connection to our SQLite database
con = sqlite3.connect("tysql.sqlite")

cur = con.cursor()

# The result of a "cursor.execute" can be iterated over by row
for row in cur.execute('PRAGMA database_list;'): # https://stackoverflow.com/questions/59709946/sqlite3-operationalerror-near-show-syntax-error
    print(row)

# Be sure to close the connection
con.close()

(0, 'main', 'C:\\Users\\GbreH\\Desktop\\SQL-training\\Data_Science_Foundation\\tysql.sqlite')


2. Виведіть перелік всіх таблиць.

3. Список всіх cust_id з таблиці Customers table.

4. Всю таблицю Customers table

5. Список клієнтів (cust_names) відсортованих від Z до A

6. Таблицю клієнтів та замовлень (cust_id і order_num) . Відсортуйте по клієнту і потім по даті замовлення

7. Таблицю (на основі Items) з кількість та вартістю товару. Відсортуйте в порядку спадання по кількості та вартості

8. Товар (з таблиці Products), ціна якого становить 9.49

9. Виведіть назву товару та ціну, яка лежить в діапазоні від  3 до 6. Відсортуйте результат в по ціні в порядку зростання

10. Кількість товару, що було продано

11. Кількість найменувань товару, ціна якого більша за 4

12. Розробіть алгоритм для виведення 3 найдорожчих товарів в базі

13. Підрахуйте кількість замовлень для кожного клієнта використовуючи підзапити

14. Виведіть список клієнтів та їх замовлення (використайте JOIN)

15. Зробіть два запити: список товарів, ціна яких менша 5 та список товарів, ціна який більша рівна 5. Використайте UNION для об'єднання цих запитів

### Завдання 4: 

Зареєструйтеся на сайті  http://www.sql-ex.ru/

Виконайте завдання із блоку SELECT; з 1 по 30 завдання

Код запитів вставляйте сюди
